# TSAI Calibration method

This method was proposed in 1987 by Tsai and consists of measuring the 3D position of 𝒏 ≥ 𝟔 control points on a 3D calibration target and the 2D coordinates of their projection in the image.

The idea of the DLT is to rewrite the Perspective Projection Equation as a homogeneous linear equation and solve it by standard methods. 

## 1. Prerequisites

The following text assumes familiarity with the camera model, which is represented in homogeneous coordinates by the following equation.

$\lambda 
\underbrace{\begin{bmatrix} u\\ v\\ 1 \end{bmatrix}}_{\bar{x}} = 
\underbrace{K \begin{bmatrix} R | T \end{bmatrix}}_{M} \cdot 
\underbrace{\begin{bmatrix} X_{W}\\ Y_{W}\\ Z_{W}\\ 1 \end{bmatrix}}_{\bar{X}}$

In this equation, the following components can be identified:

- $\lambda$: The scale factor. It is necessary to determine the depth of 3D points given the pixel coordinates (2D points).
- $\bar{x}$: The homogeneous pixel coordinates, which identify the rows ($u$) and columns ($v$) of every 2D point in the image coordinate system (where the third value is 1).
- $K$: The 3×3 matrix of intrinsic parameters, whose values are determined by the camera's physical properties.
- $[R | t]$: The 3×4 composite matrix of extrinsic parameters, where $R$ is a 3×3 rotation matrix and $t$ is a 3×1 translation vector.
- $M$: The perspective projection matrix (PPM).
- $\bar{X}$: The 4×1 vector of homogeneous world coordinates, representing the $(X, Y, Z)$ coordinates of the 3D points (where the fourth value is 1).

## 2. Assumptions and Topic's Target

This problem assumes that 2D and 3D coordinates are known, while $\lambda$ and the PPM aren't.

Therefore, the goal of the calibration process is to retrieve the '*correct*' PPM for the 2D-3D correspondence (i.e., solving the system of equations for the PPM).

The PPM estimation aims to determine the camera's intrinsic and extrinsic parameters. If only the extrinsic parameters are retrieved, the process is referred to as *camera localization*.

This method, also known as the Direct Linear Transform (DLT) algorithm, may encounter degenerate configurations caused by the arrangement of the 3D points.

Such degeneracy occurs when the 3D points:

- Lie on a plane and/or along a single line passing through the center of projection.
- Lie on a twisted cubic.



## 3. **Theory**

The goal is to isolate the unknown term and solve the system of equations for the given sets of known variables (i.e., the 2D-3D point correspondences).
In this case, the unknown term is the PPM (also referred to as M), which is composed of the intrinsic and extrinsic parameters.

Starting from the equation of the camera model expressed in homogeneous coordinates, we denote M as the matrix of intrinsic and extrinsic parameters (also referred to as the perspective projection matrix).

$ 
\begin{equation}
\begin{split}
\lambda
\begin{bmatrix}
 u\\
 v\\
 1
\end{bmatrix}
&=
\underbrace{\begin{bmatrix}
 \alpha_{u} &  0 &  u_{0}\\
 0 & \alpha_{v} &   v_{}\\
 0 & 0  & 1 \\
\end{bmatrix}}_{K}
\cdot 
\underbrace{\begin{bmatrix}
 r_{11} & r_{12} & r_{13} & t_{1} \\
 r_{21} & r_{22} & r_{32} & t_{2} \\
 r_{31} & r_{32} & r_{33} & t_{3} \\
\end{bmatrix}}_{[R|T]}
\cdot
 \begin{bmatrix}
 X_{W}\\
 Y_{W}\\
 Z_{W}\\
 1
\end{bmatrix} 
\\
\lambda
\begin{bmatrix}
 u\\
 v\\
 1
\end{bmatrix}
&=
\underbrace{\begin{bmatrix}
 m_{11} & m_{12} & m_{13} & m_{14} \\
 m_{21} & m_{22} & m_{23} & m_{24} \\
 m_{31} & m_{32} & m_{33} & m_{34} \\
\end{bmatrix}}_{PPM = M}
\cdot
 \begin{bmatrix}
 X_{W}\\
 Y_{W}\\
 Z_{W}\\
 1
\end{bmatrix} 
\end{split}
\end{equation}$

By following the next steps, the problem can be reformulated into a form that is easier to solve:
1. Re-write $M$, highlighting its three rows, $m_{1}^{T}$, $m_{1}^{T}$ and $m_{1}^{T}$.
2. Divide the third row of the system of equations (i.e., $\lambda = m_{3}^{T} \cdot P$) by the first two rows (i.e., $\lambda u = m_{1}^{T} \cdot P$ and $\lambda v = m_{2}^{T} \cdot P$).
3. Rearrange the terms of the two equations to isolate each individual value $m_{i}$ of the matrix $M$ from the rest. (Note: The scale factor $\lambda$ no longer plays a role after the second step.)

$\begin{equation}
\begin{split}
\lambda 
\begin{bmatrix} u \\ v \\ 1 \end{bmatrix} = 
\underbrace{\begin{bmatrix} m^T_1 \\ m^T_2 \\ m^T_3  \end{bmatrix}}_{M} \cdot \underbrace{\begin{bmatrix} X_W \\ Y_W \\ Z_W \\ 1\end{bmatrix}}_{P} 
& \Rightarrow
\begin{split}u ={\lambda u \over \lambda} = {m^T_1 \cdot P \over m^T_3 \cdot P} \\ v={\lambda v \over\lambda} = {m^T_2 \cdot P \over m^T_3 \cdot P}\end{split}
\\
& \Rightarrow
\begin{split} 
\big(m^T_1 - u_im^T_3 \big) \cdot P = 0 \\ \big(m^T_2 - v_im^T_3 \big) \cdot P = 0\end{split} 
\\ 
& \Rightarrow 
\begin{pmatrix}
   P^T_1 & 0^T & -u_1P^T_1\\
   0^T & P^T_1 & -v_1P^T_1
\end{pmatrix} \begin{pmatrix}m_1 \\ m_2 \\ m_3 \end{pmatrix} = \begin{pmatrix}0 \\ \vdots \\ 0\end{pmatrix} 
\end{split}
\end{equation}$

The result is a pair of equations obtained from a single 2D-3D point correspondence.

This system has 12 unknown variables. Therefore, to obtain a solution, we need at least six 2D-3D point correspondences.

By stacking the following matrices on top of one another, each obtained from a single point correspondence, 
$
\begin{pmatrix}
   P^{T}_{i} & 0^{T} & -u_{i}P^{T}_{i}\\
   0^{T} & P^{T}_{i} & -v_{i}P^{T}_{i}
\end{pmatrix}
$

we arrive at the following matrix equation:

$
\underbrace{\begin{pmatrix}
   P^T_1 & 0^T & -u_1P^T_1\\
   0^T & P^T_1 & -v_1P^T_1 \\ & \dots \\P^T_n & 0^T & -u_nP^T_n\\
   0^T & P^T_n & -v_nP^T_n 
\end{pmatrix}}_{Q\space\textbf{known}} \underbrace{\begin{pmatrix}m_{11} \\ m_{12}\\ m_{13} \\ m_{14} \\m_{21} \\ m_{22} \\ m_{23} \\ m_{24} \\ m_{31} \\ m_{32} \\ m_{33} \\ m_{34} \\ m_{41} \\ m_{42} \\ m_{43} \\ m_{44}\end{pmatrix}}_{M\space\textbf{unknown}} = \begin{pmatrix}0 \\ \vdots \\ 0\end{pmatrix}
$


[reference_coming_soon]()

**Minimal solution**

- $Q_{2n×12}$ should have rank 11 to a unique (up to scale) *non-zero* solution $M$
- because each correspondence  provides equations, then $5+{1 \over 2}$ points correspondence are needed → **6 points**

**Over-determined solution**

- For $n>6$ points, one possible solution is the **Least Square solution**, which minimizes the sum  of squared residuals, $\begin{Vmatrix}QM\end{Vmatrix}^2$, subject to the constraint $\begin{Vmatrix}M\end{Vmatrix}^2 = 1$.
- It can be solved using the SVD. The solution is the  **eigenvector** corresponding to the **smallest eigenvalue** of the matrix $Q^TQ$ (because it is the unit vector x that minimizes $\begin{Vmatrix}Qx\end{Vmatrix}^2= x^TQ^TQx$).
- Matlab function:
    - [U,S,V] = SVD(Q);
    - M = V(: , 12);
- Python function (using Numpy):
    - U, S, V_h = np.linalg.svd(Q)
    - M = np.reshape(V_h[-1, :], (3, 4))

  
Finally, to retrieve the intrinsic and extrinsic parameters, we recall that they are stored in the matrix $M$ as follows:

$M = K(R | T)$

$\begin{bmatrix}m_{11} & m_{12} & m_{13} & m_{14} \\ m_{21} & m_{22} & m_{23} & m_{24} \\ m_{31} & m_{32} & m_{33} & m_{34}\end{bmatrix} = \begin{bmatrix} \alpha_u & 0 & u_0 \\ 0 & \alpha_v & v_0 \\0 & 0 & 1\end{bmatrix} \cdot \begin{bmatrix}r_{11} & r_{12} & r_{13} & t_{1} \\ r_{21} & r_{22} & r_{23} & t_{2} \\ r_{31} & r_{32} & r_{33} & t_{3}\end{bmatrix}$

$\begin{bmatrix}m_{11} & m_{12} & m_{13} & m_{14} \\ m_{21} & m_{22} & m_{23} & m_{24} \\ m_{31} & m_{32} & m_{33} & m_{34}\end{bmatrix} = \begin{bmatrix}\alpha_ur_{11} + u_0r_{31} & \alpha_ur_{12} + u_0r_{32} & \alpha_ur_{13} + u_0r_{33} & \alpha_ut_{1} + u_0t_{3} \\ \alpha_vr_{21} + v_0r_{31}& \alpha_vr_{22} + v_0r_{32}& \alpha_vr_{23} + v_0r_{33} & \alpha_vt_{2} + v_0t_{3}\\ r_{31} & r_{32} & r_{33} &  t_{3}\end{bmatrix}$

However, notice that we are not enforcing the constraint that $\textbf{R}$ is orthogonal, i.e., $\textbf{R} \cdot  \textbf{R}^T = \textbf{I}$

we can use the so-called **QR factorization** of $M$, which decomposes  $M$ into a $R$ (orthogonal), $T$, and an upper triangular matrix (i.e., $K$)

## COMING SOON!!
4. **Code**: The Python implementation (and later, C++) of our solution.
5. **Test**: A final test to prove that the example works as expected according to our calculations.
